In [ ]:
#scikit learn tf-idf
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Load your dataset
# Assuming df is your DataFrame with columns: title, director, writer, actor1, actor2, actor3, actor4, and plot
# Adjust the file path as per your data
df = pd.read_csv("final_dataset_with_df2.csv")

# Combine data from all columns into a single feature column
df['combined_text'] = df['title'] + ' ' + df['director'] + ' ' + df['writer'] + ' ' + df['actor1'] + ' ' + df['actor2'] + ' ' + df['actor3'] + ' ' + df['actor4'] + ' ' + df['plot']

# Initialize TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer()

# Fit and transform the combined text using TF-IDF vectorizer
X = tfidf_vectorizer.fit_transform(df['combined_text'])

# Define target variable
y = df['genre']  # Specify the target column for classification

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Initialize Random Forest classifier
rf_classifier = RandomForestClassifier(n_estimators=500, max_depth=50, min_samples_split=50, min_samples_leaf=50, random_state=42)

# Train the classifier
rf_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

# Evaluate the classifier
print(classification_report(y_test, y_pred))